### マスターファイルの確認

In [ ]:
import pandas as pd
import plotly.express as px
from batch.lib.file_io import load_parquet, save_parquet,save_csv,load_csv
from batch.lib.agg_settings import (
    PATH_ASSET_PROFIT_DETAIL,
    PATH_BALANCE_DETAIL,
    PATH_ASSET_PROFIT_DETAIL_TEST,
    PATH_ASSET_PROFIT_DETAIL_TEST2,
    PATH_BALANCE_ATTRIBUTE,
    PATH_ITEM_ATTRIBUTE
)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
# 総額の表示
output_type = "トータルリターン" #資産額、トータルリターン、含み損益、実現損益、取得価格

df = load_parquet(PATH_ASSET_PROFIT_DETAIL_TEST2)
if output_type in ["資産額", "トータルリターン", "実現損益", "取得価格"]:
    df = df.groupby("date")[output_type].sum().to_frame()
else:
    df = df.groupby("date")[output_type].sum().to_frame()
df
fig = px.line(df, x=df.index, y=output_type, title="総額の表示")
fig.show()

In [ ]:
# 資産サブタイプごとの表示
output_type = "資産額" #資産額、トータルリターン、含み損益、実現損益、取得価格

df = load_parquet(PATH_ASSET_PROFIT_DETAIL_TEST2)
sub_types = df["資産サブタイプ"].unique()
df.head()
df = df.groupby(["date","資産サブタイプ"])[output_type].sum().to_frame()
df.reset_index(inplace=True)
df.head()
fig = px.line(df, x="date", y=output_type, color="資産サブタイプ", title="資産サブタイプごとの表示")
fig.show()


In [6]:
# 特定の資産サブタイプの資産名の表示
df = load_parquet(PATH_ASSET_PROFIT_DETAIL_TEST2)
output_type = "トータルリターン" #資産額、トータルリターン、含み損益、実現損益、取得価格

sub_type = "定期預金/仕組預金"
print(df["資産サブタイプ"].unique())
df = df[df["資産サブタイプ"] == sub_type]
df.head()

df = df.groupby(["date","資産名"])[output_type].sum().to_frame()
df.reset_index(inplace=True)
df.head()

fig = px.line(df, x="date", y=output_type, color="資産名", title="資産名ごとの表示")
fig.update_layout(legend_visible=False)
fig.show()


['確定拠出年金' 'ソーシャルレンディング' '住宅ローン' '円建社債' '普通預金/MRF' '暗号資産' '投資信託' '日本国債'
 '確定年金' '国内株式' '定期預金/仕組預金' 'ポイント' '現金/電子マネー' 'セキュリティートークン' '預入金' '外貨普通預金']


In [ ]:
# 特定の資産名の内容を分析
asset_name = "口座残高(クラウドバンク)"
start_date = "2025-10-01"
end_date = "2025-12-01"

df = load_parquet(PATH_ASSET_PROFIT_DETAIL_TEST2)
output_type = ["資産額", "トータルリターン", "含み損益", "実現損益", "取得価格"]

df = df[df["資産名"] == asset_name]
df = df[(df["date"] >= start_date) & (df["date"] <= end_date)]
df = df.groupby("date")[output_type].sum()
df = df.reset_index()
df = df.sort_values("date")

fig = px.line(df, x="date", y=output_type, title="資産名ごとの表示")
fig.show()

In [ ]:
# 資産過去マスターファイルの10--11月に負債を追加して保存する
import numpy as np
df = load_parquet(PATH_ASSET_PROFIT_DETAIL)
df.head()
df_add = pd.DataFrame({
    "date":pd.date_range(start="2024-10-01", end="2024-11-30"),
    "資産タイプ":"負債",
    "資産サブタイプ":"住宅ローン",
    "金融機関口座":None,
    "資産額":np.nan,
    "トータルリターン":np.nan,
    "含み損益":np.nan,
    "実現損益":np.nan,
    "取得価格":np.nan
})
df = pd.concat([df, df_add], ignore_index=True)
df.sort_values("date", inplace=True)
df.reset_index(drop=True, inplace=True)
df.to_parquet(PATH_ASSET_PROFIT_DETAIL, index=False)


In [ ]:
# ソーシャルレンディングの預入金のサブタイプをソーシャルレンディングへ
df = load_parquet(PATH_ASSET_PROFIT_DETAIL)
accounts = ["クラウドバンク", "オルタナバンク","バンカーズ","Funds"]
mask = (df["資産サブタイプ"] == "ソーシャルレンディング") & (df["金融機関口座"].isin(accounts))
#df[mask & (df["資産名"].str.contains("残高", na=False))]
df.loc[mask, "資産サブタイプ"] = "ソーシャルレンディング"
df.loc[mask].head(1000)

#save_parquet(df, PATH_ASSET_PROFIT_DETAIL)


In [ ]:
# ALTERNAの預入金を削除し、収益は個々の資産に登録する
df = load_parquet(PATH_ASSET_PROFIT_DETAIL)
mask = (df["資産サブタイプ"] == "預入金") & (df["金融機関口座"]=="ALTERNA")
df = df[~mask]

#fig = px.line(df[mask], x="date", y=["資産額","トータルリターン","含み損益","実現損益","取得価格"], title="ALTERNAの預入金")
#fig.show()

mask = (df["資産サブタイプ"] == "セキュリティートークン") & (df["資産名"]=="三重・イオンタウン鈴鹿(ALTERNA)")
df = df[~mask]

mask = (df["資産サブタイプ"] == "セキュリティートークン") & (df["金融機関口座"]=="ALTERNA")
df.loc[mask, "取得価格"] = 0.0
df.loc[mask & (df["date"]=="2025-05-03"), "実現損益"] = -4400

#save_parquet(df, PATH_ASSET_PROFIT_DETAIL)

# 2025/5/3に-4400の実現損益を資産にいれる。
# イオンタウンは消す（クラス、詳細両方）
# 口座残高は消す（クラス、詳細両方）

In [ ]:
df = load_csv(PATH_ITEM_ATTRIBUTE)
df["比率"] = pd.to_numeric(df["比率"], errors="coerce")
df["積極投資"] = pd.to_numeric(df["積極投資"], errors="coerce")
df["生活防衛資金"] = pd.to_numeric(df["生活防衛資金"], errors="coerce")

df.dtypes



### キャッシュファイルの確認


In [ ]:
import pandas as pd
import plotly.express as px
from batch.lib.file_io import load_parquet
from batch.lib.agg_settings import (
    PATH_ASSET_CACHE_DAILY,
    PATH_ASSET_CACHE_MONTHLY,
    PATH_ASSET_CACHE_YEARLY,
    PATH_CATEGORY_CACHE_DAILY,
    PATH_CATEGORY_CACHE_MONTHLY,
    PATH_CATEGORY_CACHE_YEARLY,
)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
df = load_parquet(PATH_CATEGORY_CACHE_MONTHLY)
df = df[["date", "収支_目標", "収支_実績", "収支_目標_収支タイプ_特別収支", "収支_実績_収支タイプ_特別収支"]]
df
#df.set_index("date", inplace=True)
#df.head()
#df = df.groupby("date")["資産額"].sum()
#fig = px.line(df)
#fig.show()

### グラフ描画確認

In [ ]:
# 資産ツリーマップ
import pandas as pd
import numpy as np
import plotly.graph_objects as go

fig = go.Figure(
    go.Treemap(
        labels=["資産", "株式", "日本株", "米国株", "現金", "銀行預金", "財布"],
        parents=["", "資産", "株式", "株式", "資産", "現金", "現金"],
        values=[1650, 1300, 500, 800, 350, 300, 50],  # 親も手計算
    )
)
fig.show()
